In [18]:
import fitz
import operator


In [19]:
doc = fitz.open("189641.pdf")  

In [20]:
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=operator.itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles

In [21]:
def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.
    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict
    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size

    # sorting the font sizes high to low, so that we can append the right integer to each tag 
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag

In [22]:
def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    block_string += "|"

                header_para.append(block_string)

    return header_para

In [23]:
font_counts, styles = fonts(doc)
size_tag = font_tags(font_counts, styles)
headers_para = headers_para(doc, size_tag)

In [24]:
font_counts

[('9.950390815734863', 273),
 ('11.988530158996582', 144),
 ('11.029430389404297', 58),
 ('15.944744110107422', 8),
 ('14.026607513427734', 8),
 ('8.991293907165527', 6),
 ('17.98288345336914', 1)]

In [25]:
styles

{'11.988530158996582': {'size': 11.988530158996582, 'font': 'Times-Roman'},
 '17.98288345336914': {'size': 17.98288345336914, 'font': 'Helvetica-Bold'},
 '15.944744110107422': {'size': 15.944744110107422, 'font': 'Helvetica-Bold'},
 '9.950390815734863': {'size': 9.950390815734863, 'font': 'Helvetica'},
 '11.029430389404297': {'size': 11.029430389404297, 'font': 'Helvetica'},
 '14.026607513427734': {'size': 14.026607513427734, 'font': 'Helvetica'},
 '8.991293907165527': {'size': 8.991293907165527, 'font': 'Times-Roman'}}

In [26]:
size_tag

{17.98288345336914: '<h1>',
 15.944744110107422: '<h2>',
 14.026607513427734: '<h3>',
 11.988530158996582: '<h4>',
 11.029430389404297: '<h5>',
 9.950390815734863: '<p>',
 8.991293907165527: '<s1>'}

In [27]:
headers_para

['||||',
 '<h4>- 1 - |',
 '',
 '<h1>SHEET METAL DESIGN HANDBOOK |',
 '|',
 '<h2>Forming Basics',
 '<p>……………………………………………………. 2 |',
 '|',
 '<h5>Critical Dimensions || Embosses and Offsets || Bend Radius || Bend Relief || Forming Near Holes || Form height to thickness ratio || Edge Distortion |',
 '|',
 '<h2>Laser cutting',
 '<p>……………………………………………………………5 ||',
 '<h5>Tolerances || Material Restrictions || Acceptable Materials || Localized Hardening || Hole Diameter |',
 '|',
 '<h2>CNC Turret Basics',
 '<p>………………………………………………….6 ||',
 '<h5>Tolerances || Special Forms || Hole-to-edge clearance || Hole taper || Hole diameter || Feature placement restrictions || Nibbling Large Radii || Countersinks|',
 '|',
 '<h2>Stamping',
 '<p>…………………………………………………………………….10 ||',
 '<h5>Blanking-Corners || Notches and Tabs || Cutoffs || Piercing Holes || Edge-to-Hole clearance || Forming-Bend Relief || Edge Bulging || Hole-to-form || Slot-to-form || Drawing shapes || Drawing Radii ||||',
 '||||',
 '|',
 '<h4>- 2 -